In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv


In [3]:

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

In [4]:
lines=pd.read_csv("../input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv",encoding='utf-8')

In [5]:
lines.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [6]:
lines.shape

(127607, 3)

In [7]:
lines['source'].value_counts()

tides        50000
ted          39881
indic2012    37726
Name: source, dtype: int64

In [8]:
lines=lines[lines['source']=='ted']

In [9]:
lines.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है


In [10]:
pd.isnull(lines).sum()

source              0
english_sentence    0
hindi_sentence      0
dtype: int64

In [11]:
lines.drop_duplicates(inplace=True)

In [12]:
lines=lines.sample(n=25000,random_state=42)
lines.shape

(25000, 3)

In [13]:
# Lowercase all characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())
# Remove quotes
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))
exclude = set(string.punctuation)
# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))
# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))
# Add start and end tokens to target sequences
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [14]:
lines.head()

,source,english_sentence,hindi_sentence
82040,ted,we still dont know who her parents are who she is,START_ हम अभी तक नहीं जानते हैं कि उसके मातापि...
85038,ted,no keyboard,START_ कोई कुंजीपटल नहीं _END
58018,ted,but as far as being a performer,START_ लेकिन एक कलाकार होने के साथ _END
74470,ted,and this particular balloon,START_ और यह खास गुब्बारा _END
122330,ted,and its not as hard as you think integrate cli...,START_ और जितना आपको लगता है यह उतना कठिन नहीं...


In [15]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [16]:
print(all_eng_words)

{'disconnect', 'telegrams', 'grip', 'canvas', 'lula', 'selling', 'nip', 'dawn', 'obviously', 'reminded', 'unrelated', 'liked', 'help”', 'inherently', 'overglorified', 'overlap', 'sauri', 'stirred', '“wed', 'wingspan', '“me', 'consecutive', 'vinyl', 'insecurity', 'c', 'finger', 'competent', 'crammed', 'vulnerable', 'dime', 'praise', 'traced', 'quentin', 'prostitute', 'metric', 'redesigning', 'sleepdeprived', 'van', 'canceled', 'blogs', 'individualistic', 'gamble', 'pani”', 'throwing', 'poetic', 'absolutely', 'bothered', 'africans”', 'advise', 'thrust', 'official', 'hatchet', 'pollinators', 'anyones', 'watches', 'demagogues', 'psychedelic', 'upstairs”', 'fertility', 'enterprises', 'smallpox', 'historic', 'list', 'sealed', 'trade', 'pharmaceutical', 'viral', 'movie', 'treble', 'diamond”', 'bylanes', 'unlimited', 'chess', 'handicaps', 'knows', 'bittorrent', 'bad”', 'opening', 'antenna', 'salesmen', 'elementary', 'from', 'rediscovering', 'sorts', 'ossified', 'antiseizure', 'next', 'usual', 

In [17]:
len(all_eng_words)

14030

In [18]:
print(all_hindi_words)

{'विशिष्ट्ता', 'आतें', 'उपराष्ट्रपति', 'विशेषतायें', 'जोडना', 'लिखावट', 'घुसती', 'अनीमिया', 'राक्षस', 'डैशबोर्ड', 'जाने', 'वेबसाइट', 'इस्के', 'लेतें', 'लडतेभिडते', 'ढकना', 'वजनी', 'जीवाणुओं', 'c', 'पोषक', 'घन', 'शेर', 'कैमरा', 'बनने', 'लगाने', 'अाता', 'सलमान', 'सौंप', 'ईश्वरतुल्य', 'क्रियान्वित', 'गुज़रता', 'जुङी', 'मार्क', 'नुर्सों', 'कमज़ोर', 'फैंलाना', 'चिंपैंजी', 'चढ़ा', 'स्वाभाविक', 'दिखाया।', 'महामारी', 'सरवाक', 'गाडियाँ', 'राहगीर', 'बताता', 'सकनेवाले', 'टेटू।', 'टिपर', 'बढना।', 'बनी', 'अनसुना', 'टेओडोरिन', 'बूंद', 'कबूल', 'गोलियाँ', 'बिज़नस', 'हफ्ते', 'फैसलों', 'बूंदों', 'बच्चा', 'कर्म', 'समर्थित', '“बंद', 'आविश्कार', 'अहिंसक', 'सुनिए', 'सर्वस्व', 'जरूरत', 'तकिये', 'सप्ताह', 'दृष्टिकोणों', 'मचान', 'ठहर', 'आत्महत्या', 'भेजती', 'डराने', 'अंदाज़ा', 'मस्तिष्कउत्तेजन', 'सीमायें', 'घंटे', 'हैश', 'कीमती', 'कन्स्यूम', 'बंधा', 'भगवान', 'टिक', 'स्थापना', '“तुलसी', 'व्यसन', 'औसतन', 'गयीं', 'दैवीयता', 'स्काइप', 'एकीकरण', 'घुसे', 'सोखता', 'स्वारतमोरे', 'शास्त्री', 'प्रतिफल', 'उत्तरी', 'व्यवस

In [19]:
len(all_hindi_words)

17540

In [20]:
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [21]:
lines.head()

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
82040,ted,we still dont know who her parents are who she is,START_ हम अभी तक नहीं जानते हैं कि उसके मातापि...,11,16
85038,ted,no keyboard,START_ कोई कुंजीपटल नहीं _END,2,5
58018,ted,but as far as being a performer,START_ लेकिन एक कलाकार होने के साथ _END,7,8
74470,ted,and this particular balloon,START_ और यह खास गुब्बारा _END,4,6
122330,ted,and its not as hard as you think integrate cli...,START_ और जितना आपको लगता है यह उतना कठिन नहीं...,16,20


In [22]:
#Let's consider sentences with length <= 20
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [23]:
lines.shape

(24774, 5)

In [24]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [25]:
max_length_tar=max(lines['length_hin_sentence'])
max_length_src=max(lines['length_eng_sentence'])

In [26]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
print(num_encoder_tokens, num_decoder_tokens)
num_decoder_tokens = num_decoder_tokens + 1 #for zero padding
num_encoder_tokens = num_encoder_tokens + 1 

input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

lines = shuffle(lines)
lines.head()

14030 17540


,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
20018,ted,i asked them to take the examples that we were...,START_ मैनें उनसे उन उदाहरणों को देखने को कहा ...,11,12
10239,ted,but india still has a lot of work to do,START_ मगर भारत को अभी भी बहुत काम करना है _END,10,11
65715,ted,the tweets the photos the videos the blog posts,START_ त्वीट्स फोटो वीडियो ब्लॉग पोस्ट _END,9,7
80553,ted,and i went to see “the sunday sport”,START_ और मैं “रविवार खेल” देखने गया था _END,8,9
119791,ted,the most sophisticated solar cooker,START_ अत्यंत जटिल सौरचूल्हा बनाती हैं। _END,5,7


In [27]:
X, y = lines['english_sentence'], lines['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((19819,), (4955,))

In [28]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [29]:
latent_dim=300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [30]:
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 300)    4209300     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    5262300     input_2[0][0]                    
______________________________________________________________________________________________

In [31]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 60
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
154/154 [==============================] - 77s 441ms/step - loss: 3.5661 - val_loss: 2.9138
Epoch 2/60
154/154 [==============================] - 66s 425ms/step - loss: 2.8239 - val_loss: 2.7814
Epoch 3/60
154/154 [==============================] - 66s 427ms/step - loss: 2.6496 - val_loss: 2.7118
Epoch 4/60
154/154 [==============================] - 66s 428ms/step - loss: 2.5338 - val_loss: 2.6665
Epoch 5/60
154/154 [==============================] - 66s 429ms/step - loss: 2.4405 - val_loss: 2.6288
Epoch 6/60
154/154 [==============================] - 66s 425ms/step - loss: 2.3569 - val_loss: 2.6050
Epoch 7/60
154/154 [==============================] - 66s 427ms/step - loss: 2.2739 - val_loss: 2.5867
Epoch 8/60
154/154 [==============================] - 65s 425ms/step - loss: 2.1940 - val_loss: 2.5788
Epoch 9/60
154/154 [==============================] - 65s 422ms/step - loss: 2.1203 - val_loss: 2.5670
Epoch 10/60
154/154 [==============================] - 65s 422ms/step - l

In [32]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [34]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
for k in range(3):
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('Input English sentence:', X_train[k:k+1].values[0])
    print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
    print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: the engine another
Actual Hindi Translation:  इंजन और से 
Predicted Hindi Translation:  इंजन और से _END
Input English sentence: when talking about morality we value differences of opinion
Actual Hindi Translation:  जब हम नैतिकता की चर्चा करते हैं तब हम मतभेदों को इतना अहमियत देते हैं 
Predicted Hindi Translation:  जब हम नैतिकता की चर्चा करते हैं तब हम करते जाते हैं
Input English sentence: who bragged that he had only gotten
Actual Hindi Translation:  जिन्होने बड़ी डींगें हाँकी कि 
Predicted Hindi Translation:  जिन्होने बड़ी डींगें हाँकी कि _END
